# Preprocessing data

## Normalisasi

import dataset yang sudah bersih, disini hanya diambil 1000 data random dari keseluruhan data pada dataset

In [29]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import json

In [30]:
# Membaca data JSON
data = pd.read_json("../../cleansing-data/rewind-2015/clean_comments.json")

# Mengonversi data JSON menjadi DataFrame
df = pd.json_normalize(data["clean_comments"])

# Mengambil data random sebanyak 1000 baris
df = df.sample(n=100, random_state=42)  # Ubah nilai random_state sesuai kebutuhan

df

,author,updated_at,like_count,text
15544,@ardiansyahputra03,2015-12-13T02:57:58Z,0,mana mbak sasa stepenson
12229,@TakiJoe777,2016-01-27T18:17:53Z,0,bang eka please buatin mixnya lagu bad blod ca...
12524,@washoiii,2016-01-14T12:37:41Z,0,watch me whip naenae
11361,@jasonshock1578,2016-04-04T13:26:17Z,0,10 nama lgunya apa
11037,@ajiteguh9080,2016-05-07T07:27:09Z,0,aulion yang paling hebat
...,...,...,...,...
3827,@parx4715,2020-07-13T13:19:11Z,0,dlu fun now just giveaway yg membuat indo maki...
11188,@aldealmanda2569,2016-04-19T21:00:50Z,0,awesome col
14922,@thefelipevaldes,2015-12-14T15:55:04Z,233,can wait to go to indonesia this country is am...
290,@BangAce,2023-12-09T00:49:41Z,1,masih ini yg terbaik menurut pendapat ku


In [31]:
norm = {
    " yg ": " yang ",
    " kren ": " keren",
    " sdh ": " sudah ",
    " dgn ": " dengan ",
    " tdk ": " tidak ",
    " dlm ": " dalam ",
    " dl ": " dulu ",
    " gw ": " saya ",
    " kok ": " kenapa ",
    " bs ": " bisa ",
    " bgt ": " sangat ",
    " njir ": " kagum ",
    " anjir ": " kagum ",
    " g ": " tidak ",
    " lg ": " lagi ",
    " dr ": " dari ",
    " tp ": " tapi ",
    " jg ": " juga ",
    " sm ": " sama ",
    " krn ": " karena ",
    " spt ": " seperti ",
    " yg ": " yang ",
    " pd ": " pada ",
    " dn ": " dan ",
    " d ": " di ",
    " kt ": " kita ",
    " km ": " kamu ",
    " ms ": " masih ",
    " tlg ": " tolong ",
    " sja ": " saja ",
    " sy ": " saya ",
    " sdikit ": " sedikit ",
    " dket ": " dekat ",
    " lbh ": " lebih ",
    " sj ": " saja ",
    " bnyk ": " banyak ",
    " dgnnya ": " dengannya ",
    " sgala ": " segala ",
    " skrg ": " sekarang ",
    " nmr ": " nomor ",
    " srg ": " sering ",
    " ttg ": " tentang ",
    " smua ": " semua ",
    " trus ": " terus ",
    " pdhl ": " padahal ",
    " kt ": " kita ",
    " dkt ": " dekat ",
    " jg ": " juga ",
    " dlm ": " dalam ",
    " bhw ": " bahwa ",
    " sja ": " saja ",
    " mrk ": " mereka ",
    " sgth ": " sangat ",
    " dn ": " dan ",
    " trlalu ": " terlalu ",
    " msing2 ": " masing-masing ",
    " brp ": " berapa ",
    " aq ": " aku ",
    " smpe ": " sampai ",
    " kalo ": " kalau ",
    " bkn ": " bukan ",
    " jd ": " jadi ",
    " dket ": " dekat ",
    " ntr ": " nanti ",
    " klo ": " kalau ",
    " sda ": " sudah ",
    " sy ": " saya ",
    " kpn ": " kapan ",
    " dmn ": " dimana ",
    " lgsng ": " langsung ",
    " tdk ": " tidak ",
    " pgi ": " pagi ",
    " ngga ": " tidak ",
    " gak ": " tidak ",
    " lbih ": " lebih ",
    " cm ": " cuman ",
    " sgt ": " sangat ",
    " aj ": " saja ",
    " apa2 ": " apa-apa ",
    " krg ": " kurang ",
    " ngerti ": " mengerti ",
    " slalu ": " selalu ",
    " bbrp ": " beberapa ",
    " smpe ": " sampai ",
    " krja ": " kerja ",
    " bbrp ": " beberapa ",
    " jln ": " jalan ",
    " hr ": " hari ",
    " kok ": " kenapa ",
    " mgkin ": " mungkin "
    # Tambahkan aturan normalisasi lainnya di sini sesuai kebutuhan
}


Fungsi normalisasi

In [32]:
def normalisasi(text):
    for i in norm:
        text = text.replace(i, norm[i])
    return text

In [33]:
df["text"] = df["text"].apply(lambda x: normalisasi(x))

df

,author,updated_at,like_count,text
15544,@ardiansyahputra03,2015-12-13T02:57:58Z,0,mana mbak sasa stepenson
12229,@TakiJoe777,2016-01-27T18:17:53Z,0,bang eka please buatin mixnya lagu bad blod ca...
12524,@washoiii,2016-01-14T12:37:41Z,0,watch me whip naenae
11361,@jasonshock1578,2016-04-04T13:26:17Z,0,10 nama lgunya apa
11037,@ajiteguh9080,2016-05-07T07:27:09Z,0,aulion yang paling hebat
...,...,...,...,...
3827,@parx4715,2020-07-13T13:19:11Z,0,dlu fun now just giveaway yang membuat indo ma...
11188,@aldealmanda2569,2016-04-19T21:00:50Z,0,awesome col
14922,@thefelipevaldes,2015-12-14T15:55:04Z,233,can wait to go to indonesia this country is am...
290,@BangAce,2023-12-09T00:49:41Z,1,masih ini yang terbaik menurut pendapat ku


## Stopwords

In [34]:
import Sastrawi

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

more_stop_words = ["tidak"]

stop_words = StopWordRemoverFactory().get_stop_words()
stop_words.extend(more_stop_words)

new_array = ArrayDictionary(stop_words)
stop_words_remover_new = StopWordRemover(new_array)

def stopword(str_text):
  str_text = stop_words_remover_new.remove(str_text)
  return str_text

df['text'] = df['text'].apply(lambda x:stopword(x))
df

,author,updated_at,like_count,text
15544,@ardiansyahputra03,2015-12-13T02:57:58Z,0,mana mbak sasa stepenson
12229,@TakiJoe777,2016-01-27T18:17:53Z,0,bang eka please buatin mixnya lagu bad blod ca...
12524,@washoiii,2016-01-14T12:37:41Z,0,watch me whip naenae
11361,@jasonshock1578,2016-04-04T13:26:17Z,0,10 nama lgunya apa
11037,@ajiteguh9080,2016-05-07T07:27:09Z,0,aulion paling hebat
...,...,...,...,...
3827,@parx4715,2020-07-13T13:19:11Z,0,dlu fun now just giveaway membuat indo makin h...
11188,@aldealmanda2569,2016-04-19T21:00:50Z,0,awesome col
14922,@thefelipevaldes,2015-12-14T15:55:04Z,233,can wait to go to indonesia this country is am...
290,@BangAce,2023-12-09T00:49:41Z,1,ini terbaik pendapat ku


## Tokenisasi
Memisahkan kalimat menjadi kata-kata

In [35]:
tokenized = df['text'].apply(lambda x:x.split())
tokenized

15544                        [mana, mbak, sasa, stepenson]
12229    [bang, eka, please, buatin, mixnya, lagu, bad,...
12524                            [watch, me, whip, naenae]
11361                              [10, nama, lgunya, apa]
11037                              [aulion, paling, hebat]
                               ...                        
3827     [dlu, fun, now, just, giveaway, membuat, indo,...
11188                                       [awesome, col]
14922    [can, wait, to, go, to, indonesia, this, count...
290                           [ini, terbaik, pendapat, ku]
16073                     [kayak, nya, kurang, duar, gitu]
Name: text, Length: 100, dtype: object

## Stemming
Mengubah kata berimbuhan menjadi kata dasar

In [36]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(text_cleaning):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in text_cleaning:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    print(d_clean)
    return d_clean

Proses berikut agak lama karena data besar

In [37]:
tokenized = tokenized.apply(stemming)
tokenized.to_csv("tokenized_comments.csv", index=False)

mana mbak sasa stepenson
bang eka please buatin mixnya lagu bad blod campur sambaladonya tuh bagus banget
watch me whip naenae
10 nama lgunya apa
aulion paling hebat
sory min telat 4tahun
nama lagu pas slowmotion apa
sial kaget ada yongleklangsung banting nih hp gw
masa liat ketawa sendiri kece abis
aulion hebat banget
keren banget emg bikin merinding
tahun 2016 ngajak motovloger donk
nama musik menit ke 14 apa
holy shiet just realize thats rich chiga fux 5
2016 ajak motomobi ridwan hanif sama motovloger2
baju pink tadi sopo yo wkwkwk
2020 spa msih nton mana ytrewind 2019 nya ktnya 2019 bnyak yutuber
nonton rewind 2021 langsung kesini
kangen aulion arap bayu skak
gila sampe merinding gua pas lagu where are now
produksion saya kalian
dunia entertainment indonesia di tangan kalian
untung masker kak aulion ndak miring sini
nonton 2021 mana aja
tetap jadi youtube rewind paling kesan
youtube rewind paling thebest
pingin ke masa lalutapi waktu bisa ulang
before analoger coming
ajak bang tara

In [38]:
tokenized

15544                             mana mbak sasa stepenson
12229    bang eka please buatin mixnya lagu bad blod ca...
12524                                 watch me whip naenae
11361                                   10 nama lgunya apa
11037                                  aulion paling hebat
                               ...                        
3827      dlu fun now just giveaway buat indo makin hancur
11188                                          awesome col
14922    can wait to go to indonesia this country is am...
290                                      ini baik dapat ku
16073                           kayak nya kurang duar gitu
Name: text, Length: 100, dtype: object

## Translate

import data dari file csv

In [39]:
data = pd.read_csv("tokenized_comments.csv", encoding="latin1")
data

,text
0,mana mbak sasa stepenson
1,bang eka please buatin mixnya lagu bad blod ca...
2,watch me whip naenae
3,10 nama lgunya apa
4,aulion paling hebat
...,...
95,dlu fun now just giveaway buat indo makin hancur
96,awesome col
97,can wait to go to indonesia this country is am...
98,ini baik dapat ku


In [40]:
from translate import Translator

# Inisialisasi counter
counter = 1

def convert_eng(text):
    global counter  # Menggunakan counter sebagai variabel global
    translator = Translator(to_lang="en", from_lang="id")
    translation = translator.translate(text)
    print(f"{counter}: {translation}")  # Mencetak teks terjemahan dengan nomor urut
    counter += 1  # Menambah nilai counter setiap kali fungsi dipanggil
    return translation

In [41]:
data["tweet_english"] = data["text"].apply(convert_eng)
data.to_csv("translate_comments.csv", index=False)

1: where is mbak sasa stepenson
2: bang eka please make the mix of the song bad blod mix sambaladonya is really good
3: watch me whip naenae
4: 10 what 'S the name OF the lagoons
5: the ultimate aulion
6: sory min 4years late
7: what is the name of the song pas slowmotion
8: damn surprise there is yongleklangsung banting ya hp gw
9: the look of time laughing at itself kece abis
10: aulion is really great
11: so cool emg it gives me goosebumps
12: 2016 invited motovloger donk
13: what is the name of the 14 minute music
14: holy shiet just realize thats rich chiga fux 5
15: 2016 invites motomobi ridwan hanif and motovloger2
16: pink shirt was sopo yo wkwkwk
17: 2020 spa msih nton mana ytrewind 2019 nya ktnya 2019 bnyak yutuber
18: watch rewind 2021 directly here
19: kangen aulion arap bayu check
20: crazy until I got goosebumps right where are you now
21: my productions you guys
22: indonesian entertainment world in your hands
23: fortunately, the aulion mask is not tilted here
24: watch